In [1]:
import os
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import cv2
import random
import time
from PIL import Image
import pandas as pd

In [2]:
# Library Constants
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkPoints = mp.solutions.hands.HandLandmark
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
DrawingUtil = mp.solutions.drawing_utils

In [3]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)

detector = HandLandmarker.create_from_options(options)
def convert_detection_result(detection_result, image):
        """
        Draws all the landmarks on the hand
        Args:
            image (Image): Image to draw on
            detection_result (HandLandmarkerResult): HandLandmarker detection results
        """
        # Get a list of the landmarks
        hand_landmarks_list = detection_result.hand_landmarks
        
        for idx in range(len(hand_landmarks_list)):
            #NOTE this only works for one hand
            listofpointstoreturn = []
            hand_landmarks = hand_landmarks_list[idx]

            # Save the landmarks into a NormalizedLandmarkList
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])
            # results = mp_hands.Hands.process(image = image)

            if hand_landmarks:
                for landmrk in hand_landmarks:
                    # for ids, landmrk in enumerate(hand_landmarks.landmark):
                        # print(ids, landmrk)
                    listofpointstoreturn.append(landmrk.x)
                    listofpointstoreturn.append(landmrk.y)
                    listofpointstoreturn.append(landmrk.z)
                return listofpointstoreturn
        
        return "Nothing Here"

I0000 00:00:1714956573.672878       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def returndatapoints(image):
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    # Draw the hand landmarks
    points = convert_detection_result(results, image)
    if points == "Nothing Here":
        border_size = 150
        frame = cv2.copyMakeBorder(image, top=border_size, 
                                bottom=border_size, left=border_size, 
                                right=border_size,
                                borderType=cv2.BORDER_CONSTANT, 
                                value=[0, 0, 0]
        )
        to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        results = detector.detect(to_detect) 
        points = convert_detection_result(results, frame)
    return points


In [5]:
def draw_landmarks_on_hand(image, detection_result):
    """
    Draws all the landmarks on the hand
    Args:
        image (Image): Image to draw on
        detection_result (HandLandmarkerResult): HandLandmarker detection results
    """
    # Get a list of the landmarks
    hand_landmarks_list = detection_result.hand_landmarks

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]

        # Save the landmarks into a NormalizedLandmarkList
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        # Draw the landmarks on the hand
        DrawingUtil.draw_landmarks(image,
                                    hand_landmarks_proto,
                                    solutions.hands.HAND_CONNECTIONS,
                                    solutions.drawing_styles.get_default_hand_landmarks_style(),
                                    solutions.drawing_styles.get_default_hand_connections_style())


In [72]:
"""
Main game loop. Runs until the 
user presses END_LETTER.
"""    
END_LETTER = "q"
# Create the hand detector
base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)
detector = HandLandmarker.create_from_options(options)

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
total_data_collector = []
CURRENT_LETTER = "b"
NUM_DATA = 20


# TODO: Load video
video = cv2.VideoCapture(0)
START_TIME = time.time()
# TODO: Modify loop condition  
while video.isOpened():
    # Get the current frame
    frame = video.read()[1]

    # Convert it to an RGB image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    # Convert the image to a readable format and find the hands
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    

    # Draw the hand landmarks
    draw_landmarks_on_hand(image, results)
    datapoints = returndatapoints(image)
    if datapoints != "Nothing Here":
        most_recent_actual_data = datapoints

    # Change the color of the frame back
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("Hand Tracking", image)

    #stops data collection at NUM_DATA
    if len(total_data_collector) >= NUM_DATA:
        break
    # Break the loop if the user presses 'q'
    if cv2.waitKey(50) & 0xFF == ord(END_LETTER):
        break
    if cv2.waitKey(50) & 0xFF == ord(CURRENT_LETTER):
        print(most_recent_actual_data)
        print(len(total_data_collector))
        if datapoints != "Nothing Here":
            total_data_collector.append(most_recent_actual_data)
            
            

video.release()
cv2.destroyAllWindows()
#cv2.imshow("Hand Tracking", image)



I0000 00:00:1714957643.951417       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


[0.6243408918380737, 0.661312997341156, -1.0599486444107242e-07, 0.601816713809967, 0.5831825137138367, -0.011027395725250244, 0.5930188298225403, 0.49536773562431335, -0.022078964859247208, 0.5921199321746826, 0.4217187762260437, -0.03163928538560867, 0.597649335861206, 0.36211711168289185, -0.04208081215620041, 0.6073231101036072, 0.49196967482566833, -0.03782939165830612, 0.6012228727340698, 0.3677627444267273, -0.0592992827296257, 0.5987471342086792, 0.2872157692909241, -0.0701712816953659, 0.5988141298294067, 0.22496289014816284, -0.07684710621833801, 0.6349940299987793, 0.5095779299736023, -0.04139512777328491, 0.617830216884613, 0.37126392126083374, -0.06466542184352875, 0.6117873191833496, 0.2848348021507263, -0.07545547187328339, 0.6078107357025146, 0.22006526589393616, -0.08182737231254578, 0.6600031852722168, 0.530900776386261, -0.04451385512948036, 0.6439201831817627, 0.39462530612945557, -0.06582129001617432, 0.6359649300575256, 0.30892372131347656, -0.07083970308303833, 0

In [73]:
len(total_data_collector)

20

In [74]:
#total_data_collector[2]

In [75]:
acquired_df = []
copy_of_total_data = total_data_collector
current_letter = CURRENT_LETTER
for i in range(len(copy_of_total_data)):
    copy_of_total_data[i].append(current_letter)
    acquired_df.append(copy_of_total_data[i])



In [76]:
dfcolnames = []
for i in range(1,22):
    for j in ["x","y","z"]:
        dfcolnames.append(j+str(i))
dfcolnames.append("label")

In [77]:
copy_df = acquired_df

In [78]:
#copy_df

In [79]:
# out_list = []
# for i in range(len(copy_df)):
#     out_list.append(copy_df[i][0])

In [80]:
end_df = pd.DataFrame(copy_df, columns = dfcolnames)

In [81]:
end_df

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,x19,y19,z19,x20,y20,z20,x21,y21,z21,label
0,0.624341,0.661313,-1.059949e-07,0.601817,0.583183,-0.011027,0.593019,0.495368,-0.022079,0.592120,...,0.673224,0.440771,-0.065083,0.665993,0.370999,-0.068263,0.659533,0.312512,-0.067793,b
1,0.647214,0.664757,2.507558e-08,0.625083,0.593357,-0.013345,0.611628,0.511908,-0.027872,0.605873,...,0.700261,0.448267,-0.089479,0.697347,0.378394,-0.098202,0.692939,0.318426,-0.102340,b
2,0.648761,0.719393,3.334931e-09,0.621256,0.647746,-0.014614,0.608807,0.554454,-0.026555,0.605861,...,0.683799,0.461558,-0.069514,0.676159,0.393355,-0.075306,0.669782,0.334236,-0.077179,b
3,0.638547,0.674117,-8.798942e-09,0.610101,0.599299,-0.011192,0.597650,0.521197,-0.025241,0.596043,...,0.691439,0.458209,-0.069339,0.688990,0.391675,-0.072756,0.684334,0.337936,-0.073860,b
4,0.643207,0.649326,4.976719e-08,0.610495,0.582506,-0.003922,0.596737,0.513728,-0.012808,0.596258,...,0.691438,0.440809,-0.074305,0.685758,0.373133,-0.081412,0.680299,0.311997,-0.084595,b
5,0.643515,0.782283,-7.653882e-08,0.606677,0.756059,-0.011260,0.588078,0.659261,-0.016877,0.605225,...,0.681167,0.460556,-0.056945,0.670823,0.388104,-0.065121,0.663213,0.324879,-0.068594,b
6,0.650237,0.647197,-9.424268e-08,0.636755,0.578287,-0.004585,0.628683,0.493530,-0.012428,0.625937,...,0.693666,0.431502,-0.071044,0.688288,0.367060,-0.076450,0.680200,0.311151,-0.077778,b
7,0.660741,0.675820,8.933954e-08,0.631180,0.598370,-0.011476,0.617104,0.507157,-0.023427,0.611602,...,0.695898,0.444972,-0.089309,0.686546,0.370392,-0.098145,0.679261,0.303628,-0.101863,b
8,0.551571,0.594787,-3.499272e-07,0.551498,0.558121,0.002050,0.557059,0.529906,-0.000002,0.561690,...,0.590314,0.526332,-0.021511,0.577532,0.518090,-0.019851,0.566811,0.516806,-0.016800,b
9,0.596263,0.623494,9.624091e-08,0.631309,0.557934,-0.007152,0.650635,0.467847,-0.017438,0.655403,...,0.552481,0.389610,-0.083855,0.548066,0.325270,-0.091502,0.548619,0.265343,-0.095713,b


In [82]:
end_df.to_csv('data/acquired_data/' + CURRENT_LETTER + '.csv', index=False) 
              

In [126]:
# copy_df[0]